In [1]:
import pandas as pd
args_gene_model =  pd.read_csv('/public/home/lizw/data/IMPORTANT_genome/arabidopsis/Araport11/bed/simplify_araport11.representative.gene_model.bed', sep='\t', 
                           names=['chrom', 'chromStart', 'chromEnd', 'name', 
                               'score', 'strand', 'thickStart', 'thickEnd', 
                               'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
                          )#gene model in bed12 format
args_refer_point =  'TSS'#TSS or TTS
args_focal_point = 1000   #upstream or downstream of refer_point region. eg. TSS point. positive number refer to upstream, and negative refer to downstream.
args_region_control = 2000 #the region compare with focal point for interaction calculating
args_align_merge_part = '/public/home/lizw/task/pore_c/pipeline_test_v1/V1test_order2_merge_part.csv'
args_keepall = '/public/home/lizw/task/pore_c/pipeline_test_v1/V1test_sorted_orderall_fornormailize.csv'
args_binsize = 50
args_prefix = 'total_gene_TSS_1000'
args_digest = '/public/home/lizw/task/pore_c/exp23_merge/result2/virtual_digest/DpnII_TAIR10.vd.fragments.parquet'

In [2]:
from porecplot_draw_func import chr_interval,anchor_reads,anchor_calcu

In [3]:
args_gene_model['gene_id'] = args_gene_model['name'].map(lambda x: x.split('.')[0])
gene_model_sub =  args_gene_model.loc[:,('chrom','chromStart','chromEnd','strand','thickStart','thickEnd','blockCount','blockSizes','blockStarts','gene_id')]

In [4]:
digest_df = pd.read_parquet(args_digest)

In [5]:
df_merge_part = pd.read_csv(args_align_merge_part,sep='\t')
merge_keepall_part = pd.read_csv(args_keepall,header=None,sep='\t',names=['chr','site'],converters={'chr':str, 'site':int})

In [9]:
gene_model_sub = gene_model_sub.sample(2)
for gene in gene_model_sub.itertuples():
    _,gene_chr,gene_start,gene_end,gene_strand,gene_thickStart,gene_thickEnd,gene_blockCount,temp_blockSizes,temp_blockStarts,gene_id = gene
    
    focal_point = args_focal_point
    refer_point = args_refer_point
    
    if refer_point == 'TSS':
        if gene_strand == '+':
            anchor_point = gene_start - focal_point
        else:
            anchor_point = gene_end + focal_point
    elif refer_point == 'TTS':
        if gene_strand == '+':
            anchor_point = gene_end - focal_point
        else:
            anchor_point = gene_start + focal_point
    
    anchor_df = digest_df.query('chrom == @gene_chr and start <= @anchor_point and end >= @anchor_point').loc[:,('chrom','start','end')]
    region_control = args_region_control
    region_dict = {'region_chr':gene_chr,'region_start':anchor_point-region_control,'region_end':anchor_point+region_control}
    region_df = pd.DataFrame(region_dict,index=[0])
    region_df['region_size'] = 2*region_control
    region_df['region_index'] = region_df.index
    anchor_interval_count,anchor_filter_merge = anchor_reads(anchor_df,df_merge_part,region_df,'fragment_mid',args_binsize,'and',args_prefix,False)
    anchor_binnor_merge_all_sort = anchor_calcu(merge_keepall_part,anchor_interval_count,region_df,args_binsize,args_prefix,False)

In [7]:
anchor_df

,chrom,start,end
38341,1,10329236,10329590


In [8]:
args_align_merge_part

'/public/home/lizw/task/pore_c/pipeline_test_v1/V1test_order2_merge_part.csv'